In [2]:
# Read in a WorldBuilder File

import json
import numpy as np
import pygmt

#dir = '/Users/billen/Box-Sync/Mybin'
worldbuilder_json_file = 'test_slab2wb_output.wb'

file_object = open(worldbuilder_json_file,'r')
wbdict = json.load(file_object)
print('The template worldbuilder file has been read in as a ', type(wbdict))

The template worldbuilder file has been read in as a  <class 'dict'>


In [3]:
# The details here depend on which features and models are being used, 
# but this provides a pretty detailed example of automating the process of 
# creating World Builder Files for a 2D subduction model.

# Order of Features in World Builder File is assumed to be:
    # [0] Plate A (West/North); models: temperature, composition
    # [1] Plate B (East/South); models: temperature, composition
    # [2] Slab; models: temperature, composition
# Compositional layers: crust and harzburgite -> model: uniform
# Plate temperature model: half space cooling
# Slab temperature model: mass conserving

# Directions are dist (x), depth (z), out-of-plane (y)
dy1 = -10e3  # width in out-of plane section
dy2 =  10e3
trench_dist = 3000e3

# Location (on map) information that is need for 2D cross section
# Cross Section Coordinates [[start_dist, start_depth], [end_dist, end_depth]]
start_dist = 0
start_depth = 0
end_dist = 6000e3 ### km, to be changed
end_depth = 0

ridge_distA = -1000e3  # can be outside of plate region and model region
ridge_distB = 5000e3  # or can be within

dip_direction = 0; # 0 - north or west; 1 - south or east
if dip_direction == 0:
    dip_point = [start_dist, 0]  # dips north or west, Plate B is subducting
else:
    dip_point = [end_dist, 0]  # dips south or east, Plate A is subducting

In [4]:
# Orientation of 2D cross section is determined by end points of a profiles

wbdict['cross section'] = [[start_dist,start_depth],[end_dist,end_depth]]

# Points outlining map geometry of a plate: in 2D these are rectangles
#plate A 
wbdict['features'][0]['coordinates'] = \
    [ [start_dist, dy1], [start_dist, dy2], [trench_dist, dy2], [trench_dist, dy2] ]

#plateB   
wbdict['features'][1]['coordinates'] = \
    [ [trench_dist, dy1], [trench_dist, dy2], [end_dist, dy2], [end_dist, dy2] ]

# Slab: trench location
# Slab location and orientation are defined by trench location and dip direction
wbdict['features'][2]['coordinates'] = [[trench_dist, dy1], [trench_dist, dy2]]
wbdict['features'][2]['dip point']  = dip_point

# Plate and slab velocities for half-space and mass conserving temperature
# Putting this here because think about this with ridge coordinates
spr_velA = 0.05  # m/yr
spr_velB = 0.05  # m/yr

wbdict['features'][0]['temperature models'][0]['spreading velocity'] = spr_velA
wbdict['features'][1]['temperature models'][0]['spreading velocity'] = spr_velB

# Locations of ridges needed for half space cooling models of both plates and slab
# These are start and end points on map
ridge_coordsA = [[ridge_distA, dy1], [ridge_distA, dy2]]
ridge_coordsB = [[ridge_distB, dy1], [ridge_distB, dy2]]

wbdict['features'][0]['temperature models'][0]['ridge coordinates'] = ridge_coordsA
wbdict['features'][1]['temperature models'][0]['ridge coordinates'] = ridge_coordsB


# Ridge and spreading velocity for slab depends on dip direction
if dip_direction == 0:
    ridge_for_slab = ridge_coordsB # dips north or west
    spr_vel_slab = spr_velB
else:
    ridge_for_slab = ridge_coordsA # dips south or east
    spr_vel_slab = spr_velA

wbdict['features'][2]['temperature models'][0]['plate velocity'] = spr_vel_slab
wbdict['features'][2]['temperature models'][0]['ridge coordinates'] = ridge_for_slab


In [5]:
# General Material Parameters
surface_temperature = 273 # K
bottom_temperature = -1 # use adiabatic temperature gradient

wbdict['surface temperature'] = surface_temperature
wbdict['potential mantle temperature'] = 1673 # K
wbdict['thermal expansion coefficient'] = 3.1e-5 # 1/K?
wbdict['specific heat'] = 1250. # ??
wbdict['thermal diffusivity'] = 1.0e-6 # m^2/s
wbdict['force surface temperature'] = 'true'
# Slab Mass Conserving (also uses general parameters above, so must be consistent)
wbdict['features'][2]['temperature models'][0]['density'] = 3300.0 # kg/m^3
wbdict['features'][2]['temperature models'][0]['thermal conductivity'] = 3.3 # ??

# Half-space Cooling Model and Mass COnserving: background temperature --> adiabatic or not?
wbdict['features'][2]['temperature models'][0]['adiabatic heating'] = 'true' 
wbdict['features'][0]['temperature models'][0]['top temperature'] = surface_temperature
wbdict['features'][0]['temperature models'][0]['bottom temperature'] = bottom_temperature
wbdict['features'][1]['temperature models'][0]['top temperature'] = surface_temperature
wbdict['features'][1]['temperature models'][0]['bottom temperature'] = bottom_temperature

# Other specific Mass-conserving temperature parameters
wbdict['features'][2]['temperature models'][0]['coupling depth'] = 100e3 # km
wbdict['features'][2]['temperature models'][0]['shallow dip'] = 30.0 # deg
wbdict['features'][2]['temperature models'][0]['taper distance'] = 100e3 # km

In [6]:
# Depths for calculating temperature and composition
# Assumes plates and slab have both a crust and harzburgite layer
min_depthA = -10e3 # km
min_depthB  = -10e3 # km
max_depthA = 150e3 # km
max_depthB = 150e3 # km
max_crustA = 7.5e3 # km
max_crustB = 7.5e3 # km
max_harzA = 30.0e3 # km
max_harzB = 30.0e3 # km
# Plate A
wbdict['features'][0]['min depth'] = min_depthA
wbdict['features'][0]['max depth'] = max_depthA
wbdict['features'][0]['temperature models'][0]['min depth'] = min_depthA
wbdict['features'][0]['temperature models'][0]['max depth'] = max_depthA
wbdict['features'][0]['composition models'][0]['min depth'] = min_depthA
wbdict['features'][0]['composition models'][0]['max depth'] = max_crustA
#wbdict['features'][0]['composition models'][1]['min depth'] = max_crustA
#wbdict['features'][0]['composition models'][1]['max depth'] = max_harzA

# Plate B
wbdict['features'][1]['min depth'] = min_depthB
wbdict['features'][1]['max depth'] = max_depthB
wbdict['features'][1]['temperature models'][0]['min depth'] = min_depthB
wbdict['features'][1]['temperature models'][0]['max depth'] = max_depthB
wbdict['features'][1]['composition models'][0]['min depth'] = min_depthA
wbdict['features'][1]['composition models'][0]['max depth'] = max_crustA
#wbdict['features'][1]['composition models'][1]['min depth'] = max_crustA
#wbdict['features'][1]['composition models'][1]['max depth'] = max_harzA

# Slab Mass Conserving Temperature and Composition
wbdict['features'][2]['temperature models'][0]['min distance slab top'] = -200e3 # km
wbdict['features'][2]['temperature models'][0]['max distance slab top'] = -200e3 # km

# Crustal thickness for slab depends on dip direction
if dip_direction == 0: # dips north or west
    wbdict['features'][2]['composition models'][0]['max distance slab top'] = max_crustB # km
    #wbdict['features'][2]['composition models'][1]['min distance slab top'] = max_crustB # km
    #wbdict['features'][2]['composition models'][1]['max distance slab top'] = max_harzB # km
else: # dips south or east
    wbdict['features'][2]['composition models'][0]['max distance slab top'] = max_crustA # km 
    #wbdict['features'][2]['composition models'][1]['min distance slab top'] = max_crustA # km
    #wbdict['features'][2]['composition models'][1]['max distance slab top'] = max_harzA # km


In [10]:
# Here we want to read in the segment list created for the trench perpendicular profile
# across the slab 2.0 data. 

segfile_object = open('izu_0.4d_p085_az-91.349_wb_slab_segments.json','r')
segdict = json.load(segfile_object)
print('The slab segment file has been read in as a ', type(segdict))

wbdict['features'][2]['segments'] = segdict['segments']

The slab segment file has been read in as a  <class 'dict'>


In [13]:
# How to output the modified dictionary as a json file
# Serializing json 
wbjson_object = json.dumps(wbdict, indent = 2)
  
# Writing to sample.json
with open("izu_sample_worldbuilder.json", "w") as outfile:
    outfile.write(wbjson_object)
